# CUDASTF 教程 Part 2: 后端、上下文与逻辑数据

在 Part 1 中，我们对 CUDA STF 有了初步了解，并通过一个简单的 AXPY 示例看到了 `scheduler`、`logical_data` 和 `cuda_kernel` 的基本用法。现在，我们将更详细地探讨 STF 的核心组件：后端与上下文，以及 STF 如何抽象和管理数据。

## 3. 后端与上下文 (Backends and contexts)

在 CUDASTF 中，**上下文 (context)** 是所有 API 调用的入口点。它存储了 CUDASTF 库的状态，并跟踪所有资源和依赖关系。上下文对象最终必须通过调用其 `finalize()` 方法来销毁（或者在基于 scheduler 的模型中，通常由 scheduler 的析构函数隐式处理）。

```cpp

#include <cuda/experimental/stf.cuh>
using namespace cuda::experimental::stf; // 文档为简洁通常如此声明

int main() {  
    context ctx; // <--- 创建一个上下文对象  
    // ... 创建逻辑数据 lX, lY ...  
    // ... 提交任务 ctx.task(...) ...  
    ctx.finalize(); // <--- 结束时销毁上下文并等待所有操作完成  
    return 0;  
}
```

在您提供的 `01-axpy.cu` 示例中 (Part 1)，我们使用的是 `stf::scheduler sched;`，并通过 `sched.execute([&](auto& stf_ctx){ ... });` 来提交任务。这里的 `stf_ctx` 就是在 execute 作用域内有效的上下文引用。`scheduler` 可以看作是管理一个或多个上下文并协调任务执行的更高级别的封装。

CUDASTF 目前提供了三种上下文后端：
1.  **`context` (通用上下文类型)**:
    *   默认使用 `stream_ctx` 作为后端。灵活性高，但可能有轻微运行时开销。
2.  **`stream_ctx` (基于流的上下文)**:
    *   依赖 CUDA 流和事件进行同步。任务是**即时 (eagerly)**启动的。
3.  **`graph_ctx` (基于图的上下文)**:
    *   通过 CUDA 图实现任务并行性。任务被放入 CUDA 图中，lambda 函数立即被捕获。
    *   对于重复的任务模式可能在性能上有所裨益。
    *   **重要限制**: 不允许任务内部与 CUDA 流同步 (如 `cudaStreamSynchronize`)。

### 选择和切换上下文类型

```cpp
#include <cuda/experimental/stf.cuh>
#include <cstdio> // For printf

using namespace cuda::experimental::stf;

// Dummy kernel for demonstration
__global__ void my_kernel(int *data, int val) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *data = val;
    }
}

int main() {
    int host_data = 0;

    // 1. 使用通用的 `context`，并将其赋值为一个 graph_ctx 实例
    printf("Using generic_ctx_as_graph:\n");
    context generic_ctx_as_graph = graph_ctx();
    auto l_data_g = generic_ctx_as_graph.logical_data(&host_data);
    generic_ctx_as_graph.task(l_data_g.write())->*[&](cudaStream_t s, slice<int> d_data_g){
        my_kernel<<<1,1,0,s>>>(d_data_g.data_handle(), 10);
    };
    generic_ctx_as_graph.finalize();
    printf("  generic_ctx_as_graph: host_data = %d\n", host_data);
    host_data = 0; // Reset for next context

    // 2. 静态选择基于 CUDA 流和事件的上下文
    printf("Using stream_context:\n");
    stream_ctx stream_context;
    auto l_data_s = stream_context.logical_data(&host_data);
    stream_context.task(l_data_s.write())->*[&](cudaStream_t s, slice<int> d_data_s){
        my_kernel<<<1,1,0,s>>>(d_data_s.data_handle(), 20);
    };
    stream_context.finalize();
    printf("  stream_context: host_data = %d\n", host_data);
    host_data = 0; // Reset for next context

    // 3. 静态选择基于 CUDA 图的上下文
    printf("Using graph_context:\n");
    graph_ctx graph_context;
    auto l_data_static_g = graph_context.logical_data(&host_data);
    graph_context.task(l_data_static_g.write())->*[&](cudaStream_t s, slice<int> d_data_static_g){
        my_kernel<<<1,1,0,s>>>(d_data_static_g.data_handle(), 30);
    };
    graph_context.finalize();
    printf("  graph_context: host_data = %d\n", host_data);

    printf("Context examples finished.\n");
    return 0;
}
```
**思考:**
*   对于大多数入门示例，默认的上下文行为（`stream_ctx`）已经足够。
*   当处理具有许多重复任务模式的复杂应用，或希望利用 CUDA Graphs 的低启动开销特性时，显式选择 `graph_ctx` 可能会带来性能优势。

**编译与运行示例:**
将上述代码保存为 `context_examples.cu` 在 `tutorial/stf/` 目录下，然后使用以下命令编译运行：

In [19]:
%%writefile context_examples.cu
#include <cuda/experimental/stf.cuh>
#include <cstdio> // For printf

using namespace cuda::experimental::stf;

// Dummy kernel for demonstration
__global__ void my_kernel(int *data, int val) {
    if (threadIdx.x == 0 && blockIdx.x == 0) {
        *data = val;
    }
}

int main() {
    int host_data[1];
    host_data[0] = 0;

    // 1. 使用通用的 `context`，并将其赋值为一个 graph_ctx 实例
    printf("Using generic_ctx_as_graph:\n");
    context generic_ctx_as_graph = graph_ctx();
    auto l_data_g = generic_ctx_as_graph.logical_data(host_data);
    generic_ctx_as_graph.task(l_data_g.write())->*[&](cudaStream_t s, slice<int> d_data_g){
        my_kernel<<<1,1,0,s>>>(d_data_g.data_handle(), 10);
    };
    generic_ctx_as_graph.finalize();
    printf("  generic_ctx_as_graph: host_data = %d\n", host_data[0]);
    host_data[0] = 0; // Reset for next context

    // 2. 静态选择基于 CUDA 流和事件的上下文
    printf("Using stream_context:\n");
    stream_ctx stream_context;
    auto l_data_s = stream_context.logical_data(host_data);
    stream_context.task(l_data_s.write())->*[&](cudaStream_t s, slice<int> d_data_s){
        my_kernel<<<1,1,0,s>>>(d_data_s.data_handle(), 20);
    };
    stream_context.finalize();
    printf("  stream_context: host_data = %d\n", host_data[0]);
    host_data[0] = 0; // Reset for next context

    // 3. 静态选择基于 CUDA 图的上下文
    printf("Using graph_context:\n");
    graph_ctx graph_context;
    auto l_data_static_g = graph_context.logical_data(host_data);
    graph_context.task(l_data_static_g.write())->*[&](cudaStream_t s, slice<int> d_data_static_g){
        my_kernel<<<1,1,0,s>>>(d_data_static_g.data_handle(), 30);
    };
    graph_context.finalize();
    printf("  graph_context: host_data = %d\n", host_data[0]);

    printf("Context examples finished.\n");
    return 0;
}

Overwriting context_examples.cu


In [20]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include context_examples.cu -o context_examples -arch=sm_86 -lcuda
!./context_examples

Using generic_ctx_as_graph:
  generic_ctx_as_graph: host_data = 10
Using stream_context:
  stream_context: host_data = 20
Using graph_context:
  graph_context: host_data = 30
Context examples finished.


## 4. 逻辑数据 (Logical data)

CUDASTF 将概念上的数据称为 **逻辑数据 (logical data)**。它是一个抽象的句柄，代表可能被透明地传输到或复制到 CUDASTF 任务所使用的不同**位置 (places)** 的数据 (如CPU内存、GPU内存)。

### 4.1 创建逻辑数据

当从用户提供的对象（例如，一个 `double` 数组）创建一个逻辑数据对象时，实际上是将原始数据的所有权转移给了 CUDASTF。因此，对原始数据的任何访问都应通过逻辑数据接口进行。

**示例1: 从主机数组创建逻辑数据**
```cpp
// logical_data_host_example.cu

#include <cuda/experimental/stf.cuh>
#include <cstdio> // For printf

using namespace cuda::experimental::stf;

__global__ void modify_on_device(slice<double> data_slice, double val) {
    if (threadIdx.x == 0 && blockIdx.x == 0 && data_slice.size() > 0) {
        data_slice(0) = val; // Modify the first element
    }
}

int main() {  
    context ctx; 
    const size_t N = 16;  
    double host_array_X[N]; 
    for(size_t i=0; i<N; ++i) host_array_X[i] = (double)i;
    printf("Initial host_array_X[0]: %f\n", host_array_X[0]);

    auto lX = ctx.logical_data(host_array_X);  
    lX.set_symbol("MyHostDataX");

    // 任务在设备上修改 lX
    ctx.task(lX.rw())->*[&](cudaStream_t s, slice<double> dX){
        modify_on_device<<<1,1,0,s>>>(dX, 100.0);
        printf("  Task: dX(0) potentially modified on device.\n");
    };

    ctx.finalize(); // STF将数据写回 host_array_X
    printf("After finalize, host_array_X[0]: %f\n", host_array_X[0]);
    return 0;  
}
```
**编译运行:**

In [21]:
%%writefile logical_data_host_example.cu
// logical_data_host_example.cu

#include <cuda/experimental/stf.cuh>
#include <cstdio> // For printf

using namespace cuda::experimental::stf;

__global__ void modify_on_device(slice<double> data_slice, double val) {
    if (threadIdx.x == 0 && blockIdx.x == 0 && data_slice.size() > 0) {
        data_slice(0) = val; // Modify the first element
    }
}

int main() {  
    context ctx; 
    const size_t N = 16;  
    double host_array_X[N]; 
    for(size_t i=0; i<N; ++i) host_array_X[i] = (double)i;
    printf("Initial host_array_X[0]: %f\n", host_array_X[0]);

    auto lX = ctx.logical_data(host_array_X);  
    lX.set_symbol("MyHostDataX");

    // 任务在设备上修改 lX
    ctx.task(lX.rw())->*[&](cudaStream_t s, slice<double> dX){
        modify_on_device<<<1,1,0,s>>>(dX, 100.0);
        printf("  Task: dX(0) potentially modified on device.\n");
    };

    ctx.finalize(); // STF将数据写回 host_array_X
    printf("After finalize, host_array_X[0]: %f\n", host_array_X[0]);
    return 0;  
}

Writing logical_data_host_example.cu


In [22]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include logical_data_host_example.cu -o logical_data_host_example -arch=sm_86 -lcuda
!./logical_data_host_example

Initial host_array_X[0]: 0.000000
  Task: dX(0) potentially modified on device.
After finalize, host_array_X[0]: 100.000000


**示例2 (回顾 `01-axpy.cu`): 从已在设备上的数据创建逻辑数据**
在 Part 1 的 `01-axpy.cu` (修改版，非原始的 `tutorial/stf/01-axpy.cu`) 中，如果先手动分配设备内存并拷贝数据，可以使用 `make_data_handle`：
```cpp
// (概念性代码片段)
// double *d_x_ptr, *d_y_ptr; 
// cudaMalloc(&d_x_ptr, N * sizeof(double)); 
// cudaMemcpy(d_x_ptr, h_x.data(), N * sizeof(double), cudaMemcpyHostToDevice); 

// slice<double> s_x_device(d_x_ptr, {N}); 
// auto ld_x = cuda::experimental::stf::make_data_handle<cuda::experimental::stf::device_memory>(s_x_device);
```
这种方式下，STF 知道数据已在设备上，但仍会管理其一致性。


### 4.2 数据接口 (Data interfaces)
CUDASTF 使用通用接口操作不同数据格式，通过形状 (shape)、每实例类型 (per-instance type) 和数据接口类 (data interface class) 描述。
高级用户可以定义自定义数据接口 (此处不详述)。

### 4.3 写回策略 (Write-back policy)
默认情况下，当逻辑数据销毁或上下文 `finalize()` 时，修改后的数据会写回原始主机数据实例。
可以通过 `logical_data_object.set_write_back(false)` 禁用特定逻辑数据的写回。


### 4.4 切片 (Slices)
CUDASTF 使用 `slice` (基于 `cuda::std::mdspan`) 来描述多维数组，即使它们非连续。
`slice` 对象包含指向数据的指针、维度和步长。复制 `slice` 不会复制底层数据。

**从左往右为0,1,2...n-th维度/步长**，这种和常规相反的layout可能是为了memory coalesce

**在核函数中使用 `slice` (回顾 `axpy_stf_kernel` from Part 1):**
```cpp
template <typename T>
__global__ void axpy_stf_kernel(T alpha, cuda::experimental::stf::slice<const T> x, cuda::experimental::stf::slice<T> y) {
    int tid = blockIdx.x * blockDim.x + threadIdx.x;
    int nthreads = gridDim.x * blockDim.x;
    for (int ind = tid; ind < x.size(); ind += nthreads) {
        y(ind) += alpha * x(ind); // 直接使用 slice 的 operator() 和 size()
    }
}
```

**定义多维切片 (示例 [`parallel_for_2D.cu`](./stf/parallel_for_2D.cu)):**
该示例展示了如何使用 `make_slice` 创建二维逻辑数据，并在 `parallel_for` 任务中使用它们。

In [ ]:
// Content of tutorial/stf/parallel_for_2D.cu
// You can open this file to see the full example.
// Here's a snippet of its main logic:
/*
#include <cuda/experimental/stf.cuh>

using namespace cuda::experimental::stf;

__host__ __device__ double x0(size_t i, size_t j) { return sin((double) (i - j)); }
__host__ __device__ double y0(size_t i, size_t j) { return cos((double) (i + j)); }

int main()
{
  context ctx;
  const size_t N = 16;
  double X[2 * N * 2 * N]; // Host array for X
  double Y[N * N];         // Host array for Y

  // Create 2D logical data using make_slice
  // lx represents a 2N x 2N matrix
  auto lx = ctx.logical_data(make_slice(&X[0], std::tuple{2 * N, 2 * N}, 2 * N));
  // ly represents an N x N matrix
  auto ly = ctx.logical_data(make_slice(&Y[0], std::tuple{N, N}, N));
  lx.set_symbol("X_2D");
  ly.set_symbol("Y_2D");

  // Task 1: Initialize lx on the device using parallel_for
  ctx.parallel_for(lx.shape(), lx.write())->*[=] _CCCL_DEVICE(size_t i, size_t j, auto sx) {
    sx(i, j) = x0(i, j); // sx is the device-side slice for lx
  };

  // Task 2: Compute ly based on lx and y0
  ctx.parallel_for(ly.shape(), lx.read(), ly.write())->*[=] _CCCL_DEVICE(size_t i, size_t j, auto sx, auto sy) {
    sy(i, j) = y0(i, j);
    for (size_t ii = 0; ii < 2; ii++) {
      for (size_t jj = 0; jj < 2; jj++) {
        sy(i, j) += sx(2 * i + ii, 2 * j + jj); // Access elements from sx (lx on device)
      }
    }
  };

  // Task 3: Verify results on the host
  ctx.parallel_for(exec_place::host(), ly.shape(), ly.read())
      ->*[=] __host__(size_t i, size_t j, slice<const double, 2> sy_host) {
            double expected = y0(i, j);
            for (size_t ii = 0; ii < 2; ii++) {
              for (size_t jj = 0; jj < 2; jj++) {
                expected += x0(2 * i + ii, 2 * j + jj);
              }
            }
            if (fabs(sy_host(i, j) - expected) > 0.001) {
              printf("Verification FAILED: sy(%zu, %zu) %f expect %f\n", i, j, sy_host(i, j), expected);
            }
          };

  ctx.finalize();
  printf("parallel_for_2D example finished. Check console for FAILED messages.\n");
}


In [27]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/parallel_for_2D.cu -o parallel_for_2D -arch=sm_86 -lcuda
!./parallel_for_2D

A[0] = 0.000000
A[1] = 1.000000
A[2] = 0.000000
A[3] = 0.000000
A[4] = 4.000000
A[5] = 0.200000
A[6] = 0.000000
A[7] = 0.000000
A[8] = 0.000000
A[9] = 4.100000
 sa size: 10


**slice的index顺序和常规的是反的，可以运行以下代码验证**
```cpp

  double A[5 * 2];
  slice<double, 2> sa = make_slice(&A[0], std::tuple{5, 2}, 5);
  sa(0, 0) = 0.0;
  sa(0, 1) = 0.2;
  sa(1, 0) = 1.0;
  sa(4, 1) = 4.1;
  sa(4, 0) = 4.0;
  for (size_t i = 0; i < 5 * 2; i++) {
    printf("A[%zu] = %f\n", i, A[i]);
  }
  printf(" sa size: %zu\n", sa.size());
```

output:
```bash
A[0] = 0.000000
A[1] = 1.000000
A[2] = 0.000000
A[3] = 0.000000
A[4] = 4.000000
A[5] = 0.200000
A[6] = 0.000000
A[7] = 0.000000
A[8] = 0.000000
A[9] = 4.100000
 sa size: 10
```

### 4.5 从形状定义逻辑数据 (Defining logical data from a shape)
当数据初始内容由任务填充时，可以仅从形状定义逻辑数据，STF 会在首次使用时自动分配内存。

**创建方法:**


In [30]:
%%writefile shape_definition_example.cu
#include <cuda/experimental/stf.cuh>
#include <cuda_runtime.h> // For cudaMemsetAsync
#include <iostream> // For std::cout, std::endl

using namespace cuda::experimental::stf;

int main() {  
    context ctx;

    // 定义一个包含10个整数的向量的逻辑数据，仅通过形状定义  
    auto lX_from_shape = ctx.logical_data(shape_of<slice<int>>(10));  
    lX_from_shape.set_symbol("X_from_shape");

    // 首次访问必须是 write() (或无初始值的 reduce())
    ctx.task(exec_place::current_device(), lX_from_shape.write())  
        ->*[&](cudaStream_t stream, slice<int> sX_instance) {  
            cudaMemsetAsync(sX_instance.data_handle(), 0, sX_instance.size() * sizeof(int), stream);  
            std::cout << "Task: Initialized X_from_shape on device." << std::endl;  
        };

    // 定义多维逻辑数据
    auto lY_2D_from_shape = ctx.logical_data(shape_of<slice<double, 2>>(16, 24)); // 16x24 double 矩阵
    lY_2D_from_shape.set_symbol("Y_2D_from_shape");

    ctx.task(exec_place::current_device(), lY_2D_from_shape.write())  
        ->*[&](cudaStream_t stream, slice<double, 2> sY_instance) {  
            cudaMemsetAsync(sY_instance.data_handle(), 1, sY_instance.size() * sizeof(double), stream); // Init with 1.0 pattern
            std::cout << "Task: Initialized Y_2D_from_shape on device. Extent(0): "  
                      << sY_instance.extent(0) << ", Extent(1): " << sY_instance.extent(1) << std::endl;  
        };

    ctx.finalize();  
    std::cout << "Finalized shape_definition_example." << std::endl;  
    return 0;  
}

Overwriting shape_definition_example.cu


In [33]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include shape_definition_example.cu -o shape_definition_example -arch=sm_86 -lcuda
!./shape_definition_example

Task: Initialized X_from_shape on device.
Task: Initialized Y_2D_from_shape on device. Extent(0): 16, Extent(1): 24
Finalized shape_definition_example.


**关键点:**
*   使用 `shape_of<slice<ElementType, Dimensions>>(dim1_size, ...)` 创建形状描述符。
*   对此类逻辑数据的首次访问必须是 `write()` 或`no_init`的 `reduce()`（初值会使用幺元）。

**动手试试:**
1.  **[`03-temporary-data.cu`](./stf/03-temporary-data.cu)**: 此示例演示了如何使用从形状创建的逻辑数据作为临时缓冲区。注意 `tmp` 如何在循环内创建和销毁。
2.  **[`09-dot-reduce.cu`](./stf/09-dot-reduce.cu)**: 此示例中 `lsum` 通过 `ctx.logical_data(shape_of<scalar_view<double>>())` 创建，用于累积归约结果。

**编译运行 `03-temporary-data.cu`:**

In [34]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/03-temporary-data.cu -o 03-temporary-data -arch=sm_86 -lcuda
!./03-temporary-data && echo "03-temporary-data finished successfully"

03-temporary-data finished successfully


**编译运行 `09-dot-reduce.cu`:**

In [35]:
!nvcc -std=c++17 -expt-relaxed-constexpr --extended-lambda -I../cccl/libcudacxx/include -I../cccl/cudax/include ./stf/09-dot-reduce.cu -o 09-dot-reduce -arch=sm_86 -lcuda
!./09-dot-reduce && echo "09-dot-reduce finished successfully (check assertions in code)"

09-dot-reduce finished successfully (check assertions in code)


我们已经详细学习了 STF 的后端、上下文以及逻辑数据的各种创建和使用方式。您现在应该对 STF 如何抽象数据、管理数据副本和处理数据生命周期有了更清晰的理解。

接下来，我们将进入教程的 Part 3，重点讨论 **"Tasks"** 的创建、数据依赖声明以及不同类型的任务。